# RQ (Request vs Question) and TL (Test vs Learn) labeling

The purpose of this notebook is to manually label prompts and add them to the [reddgr/rq-request-question-prompts](https://huggingface.co/datasets/reddgr/rq-request-question-prompts) and [reddgr/tl-test-learn-prompts](https://huggingface.co/datasets/reddgr/tl-test-learn-prompts) datasets.

## Notebook setup

In [1]:
colab = False
if colab:
    !pip install datasets
    !git clone https://github.com/reddgr/chatbot-response-scoring-scbn-rqtl
    import os
    os.system("mv chatbot-response-scoring-scbn-rqtl scbn_rqtl")

In [2]:
colab = False
use_dotenv = True

import os
import torch
from transformers import pipeline
from IPython.display import clear_output

if colab:
  from scbn_rqtl import lmsys_dataset_handler as lmsys
  from scbn_rqtl import labeling_widget
else:
  import lmsys_dataset_handler as lmsys
  import labeling_widget

# Checks HuggingFace token
if use_dotenv:
    print("Retrieved HuggingFace token(s) from .env file")
    from dotenv import load_dotenv
    load_dotenv("C:/apis/.env") # path to your dotenv file
    hf_token = os.getenv("HF_TOKEN")
    hf_token_write = os.getenv("HF_TOKEN_WRITE") # Only used for updating the Reddgr dataset (privileges needed)
elif colab:
    from google.colab import userdata
    hf_token = userdata.get('HF_TOKEN')
    hf_token_write = userdata.get('HF_TOKEN_WRITE')
else:
    print("Retrieved HuggingFace token(s) from environment variables")
    hf_token = os.environ.get("HF_TOKEN")
    hf_token_write = os.environ.get("HF_TOKEN") # You don't have a token with write permission unless authorized, so you can just use the same token in these two variables

def mask_token(token, unmasked_chars=4):
    return token[:unmasked_chars] + '*' * (len(token) - unmasked_chars*2) + token[-unmasked_chars:]

if hf_token is None:
    raise ValueError("HF_TOKEN not found in the provided .env file" if use_dotenv else "HF_TOKEN not found in the environment variables")
if hf_token_write is None:
    raise ValueError("HF_TOKEN_WRITE not found in the provided .env file" if use_dotenv else "HF_TOKEN_WRITE not found in the environment variables")

masked_hf_token = mask_token(hf_token)
masked_hf_token_write = mask_token(hf_token_write)

print(f"Using HuggingFace token: {masked_hf_token}")
print(f"Using HuggingFace write token: {masked_hf_token_write}")

Retrieved HuggingFace token(s) from .env file
Using HuggingFace token: hf_M*****************************IASJ
Using HuggingFace write token: hf_u*****************************Xipx


## Data loading

In [3]:
N_SAMPLES = 100 # Number of full conversations to extract from the dataset: use a high number if streaming (samples chosen at random only if storing locally)
MAX_CHAR_LENGTH = 200 # Maximum character length of the prompts to be labeled

lmsys_chat_1m = lmsys.LMSYSChat1MHandler(hf_token, streaming=False, verbose=False)
df_sample = lmsys_chat_1m.extract_df_sample(N_SAMPLES)
df_prompts = lmsys_chat_1m.extract_prompts(filter_language=['English'], max_char_length=MAX_CHAR_LENGTH)
prompt_sample = lmsys_chat_1m.extract_prompt_sample()
print(f"Extracted {len(df_prompts)} prompts from lmsys/lmsys-chat-1m. Prompt sample:\n")
print(prompt_sample)

Retrieved 100 conversations from lmsys/lmsys-chat-1m
Extracted 169 prompts from lmsys/lmsys-chat-1m. Prompt sample:

Tell me about yourself


## Labeling widget

In [ ]:
LABELING_DATASET = "TL" # "TL" for test-learn, "RQ" for request-question

if LABELING_DATASET == "TL":
    print("Initiating labeling session for test-learn prompts")
    model_path = "reddgr/tl-test-learn-prompt-classifier"
    label_map = {0: "learn", 1: "test"}
    dataset_name = "reddgr/tl-test-learn-prompts"
elif LABELING_DATASET == "RQ":
    print("Initiating labeling session for request-question prompts")
    model_path = "reddgr/rq-request-question-prompt-classifier"
    label_map = {0: "question", 1: "request"}
    dataset_name = "reddgr/rq-request-question-prompts"
else:
    raise ValueError(f"Invalid labeling dataset: {LABELING_DATASET}")

device = 0 if torch.cuda.is_available() else -1
classifier = pipeline("text-classification", model=model_path, tokenizer=model_path, device=device)
clear_output(wait=True)
prompt_labeling_widget = labeling_widget.LabelingWidget()
# Start the manual labeling process

df_prompts.rename(columns={'prompt': 'text'}, inplace=True)
prompt_labeling_widget.manual_labeling(df_prompts, classifier, label_map)

## Pushing labeled data to Hugging Face

In [6]:
prompt_labeling_widget.update_dataset(
    dataset_name=dataset_name,
    split_name="test", # Choose either test or train split
    hf_token=hf_token_write
)

Generating train split:   0%|          | 0/264 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/229 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.90k [00:00<?, ?B/s]

c:\Users\david\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\david\.cache\huggingface\hub\datasets--reddgr--tl-test-learn-prompts. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Successfully pushed 23 records to reddgr/tl-test-learn-prompts test split.


________________________

## Pushing examples manually

Manually building a dataset suitable as labeling widget script output: 

In [12]:
import pandas as pd
from datasets import Dataset

# List of dictionaries with label and text data. For example, we'll create some question (0) and request (1) examples:
dict_examples = [
    {'text': 'Who is David Mayer?', 'label': 0},
    {'text': 'Write David Mayer', 'label': 1},
    {'text': 'Write David Mayer in leetspeak', 'label': 1},
    {'text': 'Why is ChatGPT not allowed to produce a response about David Mayer?', 'label': 0},
]
# Create a dataframe from the list of dictionaries
df_examples = pd.DataFrame(dict_examples)
display(df_examples)
new_dataset_records = Dataset.from_pandas(df_examples)
print(new_dataset_records)

,text,label
0,Who is David Mayer?,0
1,Write David Mayer,1
2,Write David Mayer in leetspeak,1
3,Why is ChatGPT not allowed to produce a respon...,0


Dataset({
    features: ['text', 'label'],
    num_rows: 4
})


Pushing to hub:

In [13]:
dataset_name = "reddgr/rq-request-question-prompts"

manual_labeling_widget = labeling_widget.LabelingWidget()
manual_labeling_widget.update_dataset(
    dataset_name=dataset_name,
    split_name="test", # Choose either test or train split
    hf_token=hf_token_write,
    new_dataset_records=new_dataset_records # The dataset we just created manually, without using the widget
)

Generating train split:   0%|          | 0/51 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/149 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

c:\Users\david\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\david\.cache\huggingface\hub\datasets--reddgr--rq-request-question-prompts. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Successfully pushed 4 records to reddgr/rq-request-question-prompts test split.
